<a href="https://colab.research.google.com/github/efitzgerald763/Blood_brain_models/blob/main/ENSG00000096060_pediction_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Connects to your Google Drive so you can import files
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd

file_path = '/content/drive/My Drive/Colab Notebooks/Blood_brain_pred/ENSG00000096060_blood_brain.csv'
data = pd.read_csv(file_path)

# Set the index to the first column
data.set_index(data.columns[0], inplace=True)

Mounted at /content/drive


In [2]:
# Transpose the dataframe so each gene is a feature and each sample is a column
data_transposed = data.T

data_transposed.sample(4)

# Separate the target variable
target_row = 'ENSG00000096060'
y = data_transposed[target_row]
X = data_transposed.drop(columns=[target_row])

# Check the shapes to ensure they are as expected
print(X.shape)
print(y.shape)

(81, 18706)
(81,)


In [22]:
print(repr(X))

array([[ 3.03405882, -0.96819402,  2.17954197, ...,  2.36694902,
        -0.77987753,  6.15849995],
       [ 3.8070011 , -0.61224533,  3.64087908, ...,  1.84159704,
         0.2790302 ,  7.32233393],
       [ 3.79537299, -2.17211446,  2.65980449, ...,  2.56276484,
        -1.6279202 ,  6.67240443],
       ...,
       [ 2.73369283, -0.97929489,  1.86685639, ...,  2.61390675,
         0.03446319,  6.11528062],
       [ 3.37911512, -1.49087285,  2.94839323, ...,  2.23670283,
         1.01114867,  6.63725929],
       [ 3.35029368, -0.97792172,  3.64203891, ...,  2.24725401,
        -0.46298198,  6.67478364]])


In [30]:
np.any(np.isnan(X_scaled))  # Should be False
np.any(np.isinf(X_scaled))  # Should be False


False

In [44]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the VAE model
original_dim = X_scaled.shape[1]
intermediate_dim = 64
latent_dim = 10

# Encoder
inputs = Input(shape=(original_dim,))
h = Dense(intermediate_dim, activation='relu')(inputs)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# Decoder
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='linear')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

# VAE model
vae = Model(inputs, x_decoded_mean)

# Loss function
xent_loss = original_dim * tf.keras.losses.mean_squared_error(inputs, x_decoded_mean)
kl_loss = -0.5 * K.mean(z_log_var - K.square(z_mean) - K.exp(z_log_var) + 1, axis=-1)
vae_loss = K.mean(xent_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))

# Train the VAE
X_train, X_test = train_test_split(X_scaled, test_size=0.2, random_state=42)
vae.fit(X_train, epochs=100, batch_size=10, validation_data=(X_test, None))



Epoch 1/100
7/7 [==============================] - 2s 98ms/step - loss: 17740.4785 - val_loss: 19914.9766
Epoch 2/100
7/7 [==============================] - 0s 55ms/step - loss: 18302.6855 - val_loss: 19927.8008
Epoch 3/100
7/7 [==============================] - 0s 37ms/step - loss: 18604.2129 - val_loss: 19903.3359
Epoch 4/100
7/7 [==============================] - 0s 41ms/step - loss: 18735.0254 - val_loss: 19801.8555
Epoch 5/100
7/7 [==============================] - 0s 38ms/step - loss: 17798.3477 - val_loss: 19842.7109
Epoch 6/100
7/7 [==============================] - 0s 38ms/step - loss: 17494.0664 - val_loss: 19457.8828
Epoch 7/100
7/7 [==============================] - 0s 42ms/step - loss: 17724.9785 - val_loss: 19153.9180
Epoch 8/100
7/7 [==============================] - 0s 37ms/step - loss: 17028.6289 - val_loss: 18883.3828
Epoch 9/100
7/7 [==============================] - 0s 36ms/step - loss: 16477.8555 - val_loss: 18612.9941
Epoch 10/100
7/7 [============================

In [45]:
# Define the encoder model to get the latent representation
encoder = Model(inputs, z_mean)
X_latent = encoder.predict(X)
print(X_latent.shape)  # Should be (number_of_samples, latent_dim)


3/3 [==============================] - 0s 7ms/step
(81, 10)


In [46]:
print(Y_train_latent)

[[ 15.689438    26.845959     0.59926075 -11.96184    -14.809234
  -33.61141     18.695398   -15.169431   -12.615536   -31.20485   ]
 [ 42.754055    67.13723     25.131374    -4.68932    -10.655405
  -47.533516    27.133953   -44.12864    -14.287205   -59.317932  ]
 [ 39.4025      66.87071     21.2883     -34.418865   -16.041552
  -15.779143    -8.038779   -35.867805   -55.539803   -73.835144  ]
 [ 39.9587      47.415714     1.4067799  -35.20488    -10.944722
  -48.83126     17.814318   -40.89168     -5.1457534  -73.02159   ]
 [ 26.637852    50.549175    10.245051   -19.034674   -19.604063
  -34.391716    25.789162   -22.725407   -33.908966   -56.21167   ]
 [ 17.85808     45.440144    -6.2488923  -22.02026    -26.94512
  -36.604046    26.845697   -25.014387   -24.173822   -46.49655   ]
 [ 17.897196    53.100338    28.669956    -7.3657684  -17.072565
  -32.16342     25.526258   -18.436577   -24.42217    -46.20988   ]
 [ 26.489698    47.92831     32.49885    -29.59282      7.2570662
  -3

In [48]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Split the data
X_train_latent, X_test_latent, y_train, y_test = train_test_split(X_latent, y, test_size=0.2, random_state=42)

# Train the linear regression model
regressor = LinearRegression()
regressor.fit(X_train_latent, y_train)

# Predict and evaluate
y_pred = regressor.predict(X_test_latent)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("Mean Squared Error:", mse)
print(f'R^2 Score: {r2}')


Mean Squared Error: 0.49867569602344064
R^2 Score: 0.07913427291915687
